## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-09-02 10:02:48.440002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_df = application_df.drop(columns=['EIN','NAME'])
dropped_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
dropped_df.T.apply(lambda x: x.nunique(), axis=1)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
binning_df=dropped_df['APPLICATION_TYPE'].value_counts()
binning_df

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=[]
application_types_to_replace = list(binning_df[binning_df<=156].index)
        
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=[]
application_types_to_replace = list(binning_df[binning_df<=156].index)
        
# Replace in dataframe
for app in application_types_to_replace:
    dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
dropped_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
binning_class=dropped_df['CLASSIFICATION'].value_counts()
binning_class

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
binning_class[binning_class>=1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace= list(binning_class[binning_class<=150].index)
classifications_to_replace

['C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [10]:


# Replace in dataframe
for cls in classifications_to_replace:
    dropped_df['CLASSIFICATION'] = dropped_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
dropped_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dropped_df=pd.get_dummies(dropped_df,dtype=float)
dropped_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = dropped_df['IS_SUCCESSFUL']
X= dropped_df.drop('IS_SUCCESSFUL', axis = 1)
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 8)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train and Evaluate the Model

# MODEL 1

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu",input_dim = number_input_features))


# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                752       
                                                                 
 dense_1 (Dense)             (None, 25)                425       
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 1203 (4.70 KB)
Trainable params: 1203 (4.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 958us/step - loss: 0.5870 - accuracy: 0.7111
Epoch 2/50
804/804 [==============================] - 1s 921us/step - loss: 0.5584 - accuracy: 0.7296
Epoch 3/50
804/804 [==============================] - 1s 888us/step - loss: 0.5542 - accuracy: 0.7303
Epoch 4/50
804/804 [==============================] - 1s 872us/step - loss: 0.5516 - accuracy: 0.7307
Epoch 5/50
804/804 [==============================] - 1s 866us/step - loss: 0.5498 - accuracy: 0.7315
Epoch 6/50
804/804 [==============================] - 1s 859us/step - loss: 0.5483 - accuracy: 0.7310
Epoch 7/50
804/804 [==============================] - 1s 842us/step - loss: 0.5480 - accuracy: 0.7318
Epoch 8/50
804/804 [==============================] - 1s 835us/step - loss: 0.5469 - accuracy: 0.7312
Epoch 9/50
804/804 [==============================] - 1s 847us/step - loss: 0.5455 - accuracy: 0.7326
Epoch 10/50
804/804 [==============================] - 1s 857us/step - loss: 0.544

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5572 - accuracy: 0.7277 - 342ms/epoch - 1ms/step
Loss: 0.5571523904800415, Accuracy: 0.7276967763900757


In [18]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity1.h5')

/Users/adrienek/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# MODEL 2

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])

nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=55, activation="relu",input_dim = number_input_features))


# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=6, activation="relu"))
# Third hidden layer

nn_model2.add(tf.keras.layers.Dense(units=50, activation="relu"))

#Fourth hidden layer
nn_model2.add(tf.keras.layers.Dense(units=100, activation="relu"))

#Fifth hidden layer
nn_model2.add(tf.keras.layers.Dense(units=40, activation="relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 55)                2585      
                                                                 
 dense_4 (Dense)             (None, 6)                 336       
                                                                 
 dense_5 (Dense)             (None, 50)                350       
                                                                 
 dense_6 (Dense)             (None, 100)               5100      
                                                                 
 dense_7 (Dense)             (None, 40)                4040      
                                                                 
 dense_8 (Dense)             (None, 1)                 41        
                                                                 
Total params: 12452 (48.64 KB)
Trainable params: 12452

In [20]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 1ms/step - loss: 0.5712 - accuracy: 0.7177
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accuracy: 0.7284
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7289
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7320
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7329
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5455 - accuracy: 0.7348
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - accuracy: 0.7345
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7336
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5440 - accuracy: 0.7341
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5437 - accuracy: 0.7338

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5567 - accuracy: 0.7255 - 329ms/epoch - 1ms/step
Loss: 0.5567248463630676, Accuracy: 0.7254810333251953


In [23]:
# Export our model to HDF5 file
nn_model2.save('AlphabetSoupCharity2.h5')

# MODEL 3

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])

nn_model3 = tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=55, activation="relu",input_dim = number_input_features))


# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=6, activation="relu"))
# Third hidden layer

nn_model3.add(tf.keras.layers.Dense(units=50, activation="relu"))

#Fourth hidden layer
nn_model3.add(tf.keras.layers.Dense(units=100, activation="selu"))

#Fifth hidden layer
nn_model3.add(tf.keras.layers.Dense(units=40, activation="selu"))

#sixth hidden layer
nn_model3.add(tf.keras.layers.Dense(units=32, activation="relu"))

#seventh hidden layer

nn_model3.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 55)                2585      
                                                                 
 dense_10 (Dense)            (None, 6)                 336       
                                                                 
 dense_11 (Dense)            (None, 50)                350       
                                                                 
 dense_12 (Dense)            (None, 100)               5100      
                                                                 
 dense_13 (Dense)            (None, 40)                4040      
                                                                 
 dense_14 (Dense)            (None, 32)                1312      
                                                                 
 dense_15 (Dense)            (None, 32)               

In [25]:
# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 1ms/step - loss: 0.5723 - accuracy: 0.7194
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5550 - accuracy: 0.7269
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7287
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7319
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7296
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7306
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accuracy: 0.7326
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7323
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accuracy: 0.7336
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5452 - accuracy: 0.7319

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5540 - accuracy: 0.7298 - 385ms/epoch - 1ms/step
Loss: 0.5539554357528687, Accuracy: 0.7297959327697754


In [28]:
# Export our model to HDF5 file
nn_model3.save('AlphabetSoupCharity_optimizaion.h5')